In [92]:
import re
import jieba

f=open('pos_data.txt','r')
comment_pos=[]#存取评论

for line in f.readlines():
    line=line.strip()
    line=line.strip('-')
    p=re.compile('[’!"#$%&\'！\一：()*+,-./:;<=>?@[\\]^_`{|}~，0123456789a-zA-Z]+')#去掉标点符号数字字母等干扰项
    line=p.sub('',line)
    comment_pos.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))#为了方便转换成词序列

In [93]:
import pandas as pd
pos_data=pd.DataFrame()
pos_data['comment']=comment_pos

In [94]:
pos_data['sentiment']=len(comment_pos)*[1]

In [95]:
f=open('neg_data.txt','r')
comment_neg=[]
for line in f.readlines():
    line=line.strip()
    line=line.strip('-')
    p=re.compile('[’!"#$%&\'！\一：()*+,-./:;<=>?@[\\]^_`{|}~，0123456789a-zA-Z]+')
    line=p.sub('',line)
    comment_neg.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))

In [96]:
comment_all=comment_pos.copy()
comment_all.extend(comment_neg)
all_labels1=[1]*len(comment_pos)
all_labels1.extend([0]*len(comment_neg))
all_labels2=[0]*len(comment_pos)
all_labels2.extend([1]*len(comment_neg))


In [97]:
all_data=pd.DataFrame()#得到一个像train.csv的dataframe,方便使用nbsvm模型
all_data['comment']=comment_all
all_data['pos']=all_labels1
all_data['neg']=all_labels2

In [98]:
all_data.to_csv('stock_train.csv')

In [99]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [100]:
train = pd.read_csv('stock_train.csv')

In [103]:
train=train.dropna()

In [104]:
n = train.shape[0]
vec = TfidfVectorizer()
trn_term_doc = vec.fit_transform(train['comment'])
test_term_doc = vec.transform(train['comment'])

In [105]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [106]:
x = trn_term_doc
test_x = test_term_doc

In [107]:
import numpy as np
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [108]:
preds = np.zeros((len(train),2))
label_cols = ['pos', 'neg']
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit pos
fit neg


In [109]:
d=pd.DataFrame(preds)

In [113]:
d

,0,1
0,0.825307,0.174693
1,0.781626,0.218374
2,0.503193,0.496807
3,0.650305,0.349694
4,0.738683,0.261317
5,0.237993,0.762007
6,0.439486,0.560511
7,0.935380,0.064620
8,0.787717,0.212282
9,0.700434,0.299566


In [110]:
l=[]
for i in range(len(d[0])):
    if d[0][i]-d[1][i]=0.2:
        l.append(1)
    else:
        l.append(0)

In [111]:
train1=train['neg'].tolist()

In [112]:
count=0
for i in range(len(d[0])):
    if l[i]==train1[i]:
        count=count+1
print(count/len(d[0]))              

0.1562179706183806


In [ ]:
积极情感得分总和/人数=每天积极情绪得分